<a href="https://colab.research.google.com/github/ToghrulUTD/Web-Scraping-and-Sentiment-Classification-for-Amazon-Customer-Reviews/blob/main/Amazon_Ring_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Scrape Latest Customer Reviews for Ring Products

## Ring Video Doorbell - 1080p HD video, improved motion detection, easy installation

In [ ]:
# Scrape the latest reviews for product with given ASIN using scraperapi
asin = 'B08N5NQ869'
soup_contents = []
for page in range(1,60):
  time.sleep(20) # avoids CAPTCHA
  url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page}'
  payload = {'api_key':'bd6c22692874b6bd87ff88737c5f3d6e',
          'url': url}
  response = requests.get('http://api.scraperapi.com', params = payload)
  soup = BeautifulSoup(response.content, 'html.parser')
  soup_contents.append(soup)

In [ ]:
# Check content of the html
soup = soup_contents[2] 
soup.prettify

In [ ]:
ratings = soup.find_all('div', attrs={'class' : "a-section celwidget"})
rating = ratings[0]

title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
date = rating.find('span', attrs = {'data-hook':'review-date'}).text
try:
  verified = rating.find('span', attrs = {'class':'avp-badge'}).text
except:
  verified = 'Not verified'
star = float(rating.find('a', attrs = {'class':'a-link-normal'}).text[:3])
try:
  votes = rating.find('span', attrs={'class':'review-votes'}).text
except:
  votes = 'no vote'

prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
color = prod[1].split('Config')[0].strip()
configuration = prod[-1].strip()

print(title)
print(body)
print(date)
print(verified)
print(star)
print(votes)
print(color)
print(configuration)

Love it.
Exactly what we were looking for.
Reviewed in the United States on April 26, 2023
Not verified
5.0
no vote
Venetian Bronze
Doorbell only


In [ ]:
reviews_list = []
for page in soup_contents:
  # Extrack all user reviews for the page
  ratings = page.find_all('div', attrs={'class' : "a-section celwidget"})
  
  # Iterate over each review and extract rewiew contents
  for rating in ratings:
    title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
    body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
    date = rating.find('span', attrs = {'data-hook':'review-date'}).text
    try:
      verified = rating.find('span', attrs = {'data-hook':'avp-badge'}).text
    except:
      verified = 'Not verified'
    star = rating.find('a', attrs = {'class':'a-link-normal'}).text
    try:
      votes = rating.find('span', attrs = {'data-hook':'helpful-vote-statement'}).text
    except:
      votes = 'no vote'
    if rating.find('img').get('src'):
      img = 'Yes'
    else:
      img = 'No'
    
    prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
    color = prod[1].split('Config')[0].strip()
    configuration = prod[-1].strip()

    # Create a dictionary for the review
    rating_dict = {'title': title,
                  'body': body,
                  'date': date,
                  'status': verified,
                  'votes': votes,
                  'contains_image': img,
                  'color':color,
                  'configuration':configuration,
                  'score': star,
                  }

    # Append the dictionary review object to the list
    reviews_list.append(rating_dict)


In [ ]:
# Convert into dataframe
data = pd.DataFrame(reviews_list)
data['product'] = 'Ring Video Doorbell - 1080p HD video, improved motion detection, easy installation'
data['asin'] = 'B08N5NQ869'
# Save Data
data.to_csv('ring_video_doorbell.csv', index = False)
# Load and check data
data = pd.read_csv('ring_video_doorbell.csv')
data.head()

,title,body,date,status,votes,contains_image,color,configuration,score,product,asin
0,Love it.,Exactly what we were looking for.,"Reviewed in the United States on April 26, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,5.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
1,Son feels safer,Easy to install!,"Reviewed in the United States on April 26, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,5.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
2,Great Device - Beware of Pre-Setup,I have for Amazon Echo Dot's in my home and pu...,"Reviewed in the United States on April 25, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,4.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
3,Not bad,Should have gotten the battery one.,"Reviewed in the United States on April 25, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,3.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
4,Door ring,Must keep charging batteries,"Reviewed in the United States on April 25, 2023",Verified Purchase,no vote,Yes,Satin Nickel,Doorbell only,2.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           520 non-null    object
 1   body            520 non-null    object
 2   date            520 non-null    object
 3   status          520 non-null    object
 4   votes           520 non-null    object
 5   contains_image  520 non-null    object
 6   color           520 non-null    object
 7   configuration   520 non-null    object
 8   score           520 non-null    object
 9   product         520 non-null    object
 10  asin            520 non-null    object
dtypes: object(11)
memory usage: 44.8+ KB


## Ring Indoor Cam, Compact Plug-In HD security camera with two-way talk, Works with Alexa

In [ ]:
# Scrape the latest reviews for product with given ASIN using scraperapi
from tqdm import tqdm
asin = 'B07Q9VBYV8'
soup_contents = []
for page in tqdm(range(1,101)):
  time.sleep(20) # for not getting blocked
  url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page}'
  payload = {'api_key':'bd6c22692874b6bd87ff88737c5f3d6e',
             'url': url}
  response = requests.get('http://api.scraperapi.com', params = payload)
  soup = BeautifulSoup(response.content, 'html.parser')
  soup_contents.append(soup)

100%|██████████| 100/100 [1:08:50<00:00, 41.31s/it]


In [ ]:
# Check content of single review using soup object
soup = soup_contents[0] 
ratings = soup.find_all('div', attrs={'class' : "a-section celwidget"})
rating = ratings[0]

title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
date = rating.find('span', attrs = {'data-hook':'review-date'}).text
try:
  verified = rating.find('span', attrs = {'class':'avp-badge'}).text
except:
  verified = 'Not verified'
star = float(rating.find('a', attrs = {'class':'a-link-normal'}).text[:3])
try:
  votes = rating.find('span', attrs={'class':'review-votes'}).text
except:
  votes = 'no vote'

prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
style = prod[1].split('Size')[0].strip()
size = prod[2].split('Color')[0].strip()
color = prod[-1].strip()


print(title)
print(body)
print(date)
print(verified)
print(star)
print(votes)
print(style)
print(size)
print(color)

Did T work with my Wi-Fi
For some reason it wouldn’t connect to my Wi-Fi.
Reviewed in the United States 🇺🇸 on April 25, 2023
Not verified
3.0
no vote
Device Only
1 Camera
White


In [ ]:
reviews_list = []
for page in soup_contents:
  # Extrack all user reviews for the page
  ratings = page.find_all('div', attrs={'class' : "a-section celwidget"})
  
  # Iterate over each review and extract rewiew contents
  for rating in ratings:
    title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
    body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
    date = rating.find('span', attrs = {'data-hook':'review-date'}).text
    try:
      verified = rating.find('span', attrs = {'data-hook':'avp-badge'}).text
    except:
      verified = 'Not verified'
    star = rating.find('a', attrs = {'class':'a-link-normal'}).text
    try:
      votes = rating.find('span', attrs = {'data-hook':'helpful-vote-statement'}).text
    except:
      votes = 'no vote'
    if rating.find('img').get('src'):
      img = 'Yes'
    else:
      img = 'No'
    
    prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
    style = prod[1].split('Size')[0].strip()
    size = prod[2].split('Color')[0].strip()
    color = prod[-1].strip()

    # Create a dictionary for the review
    rating_dict = {'title': title,
                  'body': body,
                  'date': date,
                  'status': verified,
                  'votes': votes,
                  'contains_image': img,
                  'style': style,
                  'color':color,
                  'size':size,
                  'score': star,
                  }

    # Append the dictionary review object to the list
    reviews_list.append(rating_dict)


In [ ]:
# Convert into dataframe
data = pd.DataFrame(reviews_list)
data['product'] = 'Ring Indoor Cam, Compact Plug-In HD security camera with two-way talk'
data['asin'] = 'B07Q9VBYV8'
# Save Data
data.to_csv('ring_indoor_security_cam.csv', index = False)
# Load and check data
data = pd.read_csv('ring_indoor_security_cam.csv')
data.head()

,title,body,date,status,votes,contains_image,style,color,size,score,product,asin
0,Did T work with my Wi-Fi,For some reason it wouldn’t connect to my Wi-Fi.,"Reviewed in the United States 🇺🇸 on April 25, ...",Verified Purchase,no vote,Yes,Device Only,White,1 Camera,3.0 out of 5 stars,"Ring Indoor Cam, Compact Plug-In HD security c...",B07Q9VBYV8
1,Amazing,"So far, I am loving these indoor cameras. They...","Reviewed in the United States 🇺🇸 on April 25, ...",Verified Purchase,no vote,Yes,Device Only,Black,1 Camera,4.0 out of 5 stars,"Ring Indoor Cam, Compact Plug-In HD security c...",B07Q9VBYV8
2,Perfect camera for mom's nursing home room,"Works as advertised, easy setup-especially if ...","Reviewed in the United States 🇺🇸 on April 25, ...",Verified Purchase,no vote,Yes,Device Only,White,1 Camera,5.0 out of 5 stars,"Ring Indoor Cam, Compact Plug-In HD security c...",B07Q9VBYV8
3,Times out after 10 minutes and must be manuall...,Every aspect of this product is great except t...,"Reviewed in the United States 🇺🇸 on April 25, ...",Verified Purchase,no vote,Yes,Device Only,Black,1 Camera,1.0 out of 5 stars,"Ring Indoor Cam, Compact Plug-In HD security c...",B07Q9VBYV8
4,OK video quality,"At this point, image quality should be MUCH be...","Reviewed in the United States 🇺🇸 on April 24, ...",Verified Purchase,no vote,Yes,Device Only,White,2 Cameras,3.0 out of 5 stars,"Ring Indoor Cam, Compact Plug-In HD security c...",B07Q9VBYV8


In [ ]:
data.describe(include = 'all')

,title,body,date,status,votes,contains_image,style,color,size,score,product,asin
count,720,718,720,720,720,720,720,720,720,720,720,720
unique,646,715,205,2,19,1,2,2,2,6,1,1
top,Great,Easy setup,"Reviewed in the United States on October 6, 2022",Verified Purchase,no vote,Yes,Device Only,White,1 Camera,5.0 out of 5 stars,"Ring Indoor Cam, Compact Plug-In HD security c...",B07Q9VBYV8
freq,7,2,11,672,527,720,713,558,617,426,720,720


## Outdoor Camera: Ring Spotlight Cam Battery HD Security Camera with Built Two-Way Talk and a Siren Alarm

In [ ]:
# Scrape the latest reviews for product with given ASIN using scraperapi
from tqdm import tqdm
asin = 'B0758L64L9'
soup_contents = []
for page in tqdm(range(1,75)):
  time.sleep(20) # helps avoid CAPTCHA
  url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page}'
  payload = {'api_key':'bd6c22692874b6bd87ff88737c5f3d6e',
          'url': url}
  response = requests.get('http://api.scraperapi.com', params = payload)
  soup = BeautifulSoup(response.content, 'html.parser')
  soup_contents.append(soup)

100%|██████████| 74/74 [41:59<00:00, 34.05s/it]


In [ ]:
# Check content of the html
soup = soup_contents[0] 
ratings = soup.find_all('div', attrs={'class' : "a-section celwidget"})
rating = ratings[0]

title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
date = rating.find('span', attrs = {'data-hook':'review-date'}).text
try:
  verified = rating.find('span', attrs = {'class':'avp-badge'}).text
except:
  verified = 'Not verified'
star = rating.find('a', attrs = {'class':'a-link-normal'}).text[:3]
try:
  votes = rating.find('span', attrs={'class':'review-votes'}).text
except:
  votes = 'no vote'

prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
color = prod[1].split('Size')[0].strip()
size = prod[2].split('Configuration')[0].strip()
configuration = prod[-1].strip()


print(title)
print(body)
print(date)
print(verified)
print(star)
print(votes)
print(configuration)
print(size)
print(color)

Safety comes first
I used this for my backyard protection. It was easy to install. Instructions were step by step. Very clear picture even at night. Very pleased with this camera!
Reviewed in the United States 🇺🇸 on March 30, 2023
Not verified
VIN
no vote
Device Only
1 Cam
White


In [ ]:
reviews_list = []
for page in soup_contents:
  try:
    # Extrack all user reviews for the page
    ratings = page.find_all('div', attrs={'class' : "a-section celwidget"})
    
    # Iterate over each review and extract rewiew contents
    for rating in ratings:
      title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
      body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
      date = rating.find('span', attrs = {'data-hook':'review-date'}).text
      try:
        verified = rating.find('span', attrs = {'data-hook':'avp-badge'}).text
      except:
        verified = 'Not verified'
      star = rating.find('a', attrs = {'class':'a-link-normal'}).text
      try:
        votes = rating.find('span', attrs = {'data-hook':'helpful-vote-statement'}).text
      except:
        votes = 'no vote'
      if rating.find('img').get('src'):
        img = 'Yes'
      else:
        img = 'No'
      
      prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
      color = prod[1].split('Size')[0].strip()
      size = prod[2].split('Configuration')[0].strip()
      configuration = prod[-1].strip()

      # Create a dictionary for the review
      rating_dict = {'title': title,
                    'body': body,
                    'date': date,
                    'status': verified,
                    'votes': votes,
                    'contains_image': img,
                    'configuration': configuration,
                    'color':color,
                    'size':size,
                    'score': star,
                    }

      # Append the dictionary review object to the list
      reviews_list.append(rating_dict)
  except:
    continue

In [ ]:
# Convert into dataframe
data = pd.DataFrame(reviews_list)
data['product'] = 'Ring Spotlight Cam Battery HD Security Camera with Built Two-Way Talk and a Siren Alarm'
data['asin'] = 'B0758L64L9'
# Save Data
data.to_csv('ring_outdoor_security_cam.csv', index = False)
# Load and check data
data = pd.read_csv('ring_outdoor_security_cam.csv')
data.head(3)

,title,body,date,status,votes,contains_image,configuration,color,size,score,product,asin
0,Problem,The camera received was poor quality and batte...,"Reviewed in the United States 🇺🇸 on April 21, ...",Verified Purchase,no vote,Yes,Device Only,White,1 Cam,2.0 out of 5 stars,Ring Spotlight Cam Battery HD Security Camera ...,B0758L64L9
1,Lasted 10 months,Got this in Jan 2022 and it died Oct 2022. I c...,"Reviewed in the United States 🇺🇸 on April 17, ...",Verified Purchase,no vote,Yes,Device Only,Black,1 Cam,2.0 out of 5 stars,Ring Spotlight Cam Battery HD Security Camera ...,B0758L64L9
2,If you change your Wi-Fi camera may never conn...,I had no issues with my camera other than the ...,"Reviewed in the United States 🇺🇸 on April 17, ...",Verified Purchase,no vote,Yes,Device Only,Black,1 Cam,3.0 out of 5 stars,Ring Spotlight Cam Battery HD Security Camera ...,B0758L64L9


## Ring Smart Lighting – Bridge

In [ ]:
# Scrape the latest reviews for product with given ASIN using scraperapi
from tqdm import tqdm
asin = 'B07KXBX65F'
soup_contents = []
for page in tqdm(range(1,55)):
  time.sleep(15) # avoids CAPTCHA
  url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page}'
  payload = {'api_key':'bd6c22692874b6bd87ff88737c5f3d6e',
            'url': url}
  response = requests.get('http://api.scraperapi.com', params = payload)
  soup = BeautifulSoup(response.content, 'html.parser')
  soup_contents.append(soup)

100%|██████████| 54/54 [33:38<00:00, 37.37s/it]


In [ ]:
# Check content of the html
soup = soup_contents[0]
ratings = soup.find_all('div', attrs={'class' : "a-section celwidget"})
rating = ratings[9]

title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
date = rating.find('span', attrs = {'data-hook':'review-date'}).text
try:
  verified = rating.find('span', attrs = {'class':'avp-badge'}).text
except:
  verified = 'Not verified'
star = rating.find('a', attrs = {'class':'a-link-normal'}).text[:3]
try:
  votes = rating.find('span', attrs={'class':'review-votes'}).text
except:
  votes = 'no vote'

print(title)
print(body)
print(date)
print(verified)
print(star)
print(votes)

Great idea
Great buy
Reviewed in the United States on January 12, 2023
Not verified
5.0
no vote


In [ ]:
reviews_list = []
for page in soup_contents:
  # Extrack all user reviews for the page
  ratings = page.find_all('div', attrs={'class' : "a-section celwidget"})
  
  # Iterate over each review and extract rewiew contents
  for rating in ratings:
    title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
    body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
    date = rating.find('span', attrs = {'data-hook':'review-date'}).text
    try:
      verified = rating.find('span', attrs = {'data-hook':'avp-badge'}).text
    except:
      verified = 'Not verified'
    star = rating.find('a', attrs = {'class':'a-link-normal'}).text
    try:
      votes = rating.find('span', attrs = {'data-hook':'helpful-vote-statement'}).text
    except:
      votes = 'no vote'

    # Create a dictionary for the review
    rating_dict = {'title': title,
                  'body': body,
                  'date': date,
                  'status': verified,
                  'votes': votes,
                  'score': star
                  }

    # Append the dictionary review object to the list
    reviews_list.append(rating_dict)

In [ ]:
# Convert into dataframe
data = pd.DataFrame(reviews_list)
data['product'] = 'Ring Smart Lighting – Bridge'
data['asin'] = 'B07KXBX65F'
# Save Data
data.to_csv('ring_smart_lighting_bridge.csv', index = False)
# Load and check data
data = pd.read_csv('ring_smart_lighting_bridge.csv')
data.head(3)

,title,body,date,status,votes,score,product,asin
0,Aún a la distancia,"Funciona perfecto, le agregué una cámara y me ...","Reviewed in the United States 🇺🇸 on April 24, ...",Verified Purchase,no vote,5.0 out of 5 stars,Ring Smart Lighting – Bridge,B07KXBX65F
1,Can’t connect or use the bridge as a bridge,Can’t get this to connect. With the cold weath...,"Reviewed in the United States 🇺🇸 on April 18, ...",Verified Purchase,no vote,1.0 out of 5 stars,Ring Smart Lighting – Bridge,B07KXBX65F
2,"reliable, easy, great customer support","reliable, easy, great customer support","Reviewed in the United States 🇺🇸 on April 17, ...",Verified Purchase,no vote,5.0 out of 5 stars,Ring Smart Lighting – Bridge,B07KXBX65F


In [ ]:
data.describe()

,title,body,date,status,votes,score,product,asin
count,339,340,340,340,340,340,340,340
unique,316,333,264,2,12,6,1,1
top,Works great,Works great,"Reviewed in the United States on November 26, ...",Verified Purchase,no vote,5.0 out of 5 stars,Ring Smart Lighting – Bridge,B07KXBX65F
freq,8,3,6,323,179,168,340,340


## Ring Alarm 8-piece kit (2nd Gen) – home security system with optional 24/7 professional monitoring

In [ ]:
# Scrape the latest reviews for product with given ASIN using scraperapi
from tqdm import tqdm
asin = 'B07ZPMCW64'
soup_contents = []
for page in tqdm(range(1,60)):
  time.sleep(20) # avoids CAPTCHA
  url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page}'
  payload = {'api_key':'bd6c22692874b6bd87ff88737c5f3d6e',
            'url': url}
  response = requests.get('http://api.scraperapi.com', params = payload)
  soup = BeautifulSoup(response.content, 'html.parser')
  soup_contents.append(soup)

100%|██████████| 59/59 [36:43<00:00, 37.34s/it]


In [ ]:
# Check content of the html
soup = soup_contents[0] 
ratings = soup.find_all('div', attrs={'class' : "a-section celwidget"})
rating = ratings[1]

title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
date = rating.find('span', attrs = {'data-hook':'review-date'}).text
try:
  verified = rating.find('span', attrs = {'class':'avp-badge'}).text
except:
  verified = 'Not verified'
star = rating.find('a', attrs = {'class':'a-link-normal'}).text[:3]
try:
  votes = rating.find('span', attrs={'class':'review-votes'}).text
except:
  votes = 'no vote'

prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
style = prod[1].strip()

print(title)
print(body)
print(date)
print(verified)
print(star)
print(votes)
print(style)

Good
The system is amazing but the noise is not loud enough for my neighbors to hear
Reviewed in the United States 🇺🇸 on April 21, 2023
Not verified
3.0
no vote
Ring Alarm


In [ ]:
reviews_list = []
for page in soup_contents:
  try:
    # Extrack all user reviews for the page
    ratings = page.find_all('div', attrs={'class' : "a-section celwidget"})
    # Iterate over each review and extract rewiew contents
    for rating in ratings:
      title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
      body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
      date = rating.find('span', attrs = {'data-hook':'review-date'}).text
      try:
        verified = rating.find('span', attrs = {'data-hook':'avp-badge'}).text
      except:
        verified = 'Not verified'
      star = rating.find('a', attrs = {'class':'a-link-normal'}).text
      try:
        votes = rating.find('span', attrs = {'data-hook':'helpful-vote-statement'}).text
      except:
        votes = 'no vote'
      prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
      style = prod[1].strip()

      # Create a dictionary for the review
      rating_dict = {'title': title,
                    'body': body,
                    'date': date,
                    'status': verified,
                    'votes': votes,
                    'score': star,
                    'style': style,
                    }

      # Append the dictionary review object to the list
      reviews_list.append(rating_dict)
  except:
    continue

In [ ]:
# Convert into dataframe
data = pd.DataFrame(reviews_list)
data['product'] = 'Ring Alarm 8-piece kit (2nd Gen) – home security system with optional 24/7 professional monitoring'
data['asin'] = 'B07ZPMCW64'
# Save Data
data.to_csv('ring_alarm_kit.csv', index = False)
# Load and check data
data = pd.read_csv('ring_alarm_kit.csv')
data.head(3)

,title,body,date,status,votes,score,style,product,asin
0,Bait & Switch,"I would love this product, except for the fact...","Reviewed in the United States 🇺🇸 on April 24, ...",Verified Purchase,no vote,2.0 out of 5 stars,Ring Alarm,Ring Alarm 8-piece kit (2nd Gen) – home securi...,B07ZPMCW64
1,Good,The system is amazing but the noise is not lou...,"Reviewed in the United States 🇺🇸 on April 21, ...",Verified Purchase,no vote,3.0 out of 5 stars,Ring Alarm,Ring Alarm 8-piece kit (2nd Gen) – home securi...,B07ZPMCW64
2,Great purchase,"Easy to install and after weeks of use, still ...","Reviewed in the United States 🇺🇸 on April 15, ...",Verified Purchase,no vote,5.0 out of 5 stars,Ring Alarm,Ring Alarm 8-piece kit (2nd Gen) – home securi...,B07ZPMCW64
